In [31]:
import pandas as pd
import openpyxl
from openpyxl.styles import PatternFill, Border, Side
import os
import nbimporter

from sap_data_prepare import sap_data
from polon_data_prepare import polon_data
from merge_and_save import techers_compare, not_techers_compare, save_to_excel
from styles_changes import worksheet_style

import warnings
warnings.simplefilter("ignore")

In [32]:
#Get current and parent directory directory
current_path = os.getcwd()
parent_directory = os.path.dirname(current_path)

#Enter the file name in the raw folder, create path to SAP data
sap_data_file_name = 'SAP.xlsx'
path_sap = os.path.join(parent_directory, 'data', 'raw', sap_data_file_name)

#Enter the file name in the raw folder, create path to POLON data
polon_data_file_name = 'POLON.xlsx'
path_polon = os.path.join(parent_directory, 'data', 'raw', polon_data_file_name)


#Define the paths to save temporary SAP and POLON Excel file
path_temp_excel = os.path.join(parent_directory, 'data', 'interim',
                               polon_data_file_name.split('.')[0] + '_VS_' + sap_data_file_name)


#Define the paths to save final SAP and POLON Excel file
path_final_excel = os.path.join(parent_directory, 'data', 'processed',
                                polon_data_file_name.split('.')[0] + '_VS_' + sap_data_file_name)

In [27]:
def main():
    #Extract data for teachers and non-teachers
    sap_t, sap_nt = sap_data(path_sap) #Extract data from SAP, definition from 'sap_data_prepare' notebook
    polon_t, polon_nt = polon_data(path_polon) #Extract data from POLON, definition from 'polon_data_prepare' notebook
    
    #Compare SAP and POLON data, definitions from 'merge_and_save' notebook
    t_compare = techers_compare(sap_t, polon_t)
    nt_compare = not_techers_compare(sap_nt, polon_nt)
    
    #Save the comparison results to a temporary Excel file, definition from 'merge_and_save' notebook
    save_to_excel(t_compare, nt_compare, path_temp_excel)
    
    #Load the temporary Excel file and access the worksheets
    wb = openpyxl.load_workbook(path_temp_excel, data_only=True)
    ws_t = wb['teachers']
    ws_nt = wb['not_teachers']

    #Apply formatting, definition from 'styles_changes' notebook
    worksheet_style(ws_t)
    worksheet_style(ws_nt)

    #Save the formatted Excel file to the final destination
    wb.save(path_final_excel)
    wb.close()
    
#Run the main function    
if __name__ == "__main__":
    main()